# Preprocess Datasets for Korean LLM (Large Language Model) fine-tuning
---

- Alpaca 논문에서 전처리했던 방식대로 전처리 수행
- 허깅페이스 인증 정보 설정: `huggingface-cli login`
    - https://huggingface.co/join
    - https://huggingface.co/settings/tokens

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates')

from common_lib import check_packages
check_packages()

++++++++++++++++++++++++++++++++++++++++++++++++++++++++
[ERROR] 0번 모듈 노트북(0_setup.ipynb)을 먼저 실행해 주세요.
++++++++++++++++++++++++++++++++++++++++++++++++++++++++


<br>

## 1. Download LLM from Hugging Face hub
---

### Load dataset
허깅페이스 허브에서 다운로드하거나 json/json 포맷의 데이터 세트를 다운로드합니다. 데이터 세트 내 샘플은 (`instruction, input, output`)의 
key-value나 (`instruction, output`)의 key-value로 구성되어야 합니다.

구름 데이터셋 v2는 GPT-4-LLM, Vicuna, 그리고 Databricks의 Dolly 데이터셋을 병합한 것입니다. 이 모든 데이터셋은 DeepL을 이용하여 한국어로 번역되었습니다.
https://huggingface.co/datasets/nlpai-lab/kullm-v2

예시:
```
{
    "instruction":"건강을 유지하기 위한 세 가지 팁을 알려주세요.",
    "input":"",
    "output":"세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다."
}
```

In [2]:
import os
import torch
import transformers
from datasets import load_dataset
from local_utils.inference_lib import Prompter
from transformers import GPTNeoXForCausalLM, GPTNeoXTokenizerFast

data_path = "nlpai-lab/kullm-v2"
#data_path = "beomi/KoAlpaca-v1.1a"
#data_path = "./data/ko_alpaca_data.json"

if data_path.endswith(".json") or data_path.endswith(".jsonl"):
    data = load_dataset("json", data_files=data_path)
else:
    data = load_dataset(data_path)
    
prompter = Prompter("kullm")
cutoff_len = 2048
train_on_inputs = True

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
data_path

'nlpai-lab/kullm-v2'

In [4]:
import os
from pathlib import Path
from huggingface_hub import snapshot_download

HF_MODEL_ID = "nlpai-lab/kullm-polyglot-12.8b-v2"

tokenizer = GPTNeoXTokenizerFast.from_pretrained(HF_MODEL_ID)

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

# create model dir
model_name = HF_MODEL_ID.split("/")[-1].replace('.', '-')
model_tar_dir = Path(f"/home/ec2-user/SageMaker/models/{model_name}")
if not os.path.isdir(model_tar_dir):
    os.makedirs(model_tar_dir, exist_ok=True)
    # Download model from Hugging Face into model_dir
    snapshot_download(
        HF_MODEL_ID, 
        local_dir=str(model_tar_dir), 
        local_dir_use_symlinks=False,
        allow_patterns=allow_patterns,
        cache_dir="/home/ec2-user/SageMaker/"
    )

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'GPTNeoXTokenizerFast'.
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

# 입력 데이터셋 살펴보기 EDA

데이터 구조
- id: 각 데이터 포인트의 고유 식별자
- instruction: 사용자의 질문 또는 명령
- input: 추가 입력 (여기서는 비어 있음)
- output: 질문 또는 명령에 대한 답변

In [5]:
# 데이터셋의 처음 3개 샘플을 출력
print(data['train'][:3])

{'id': ['alpaca_{idx}', 'alpaca_{idx}', 'alpaca_{idx}'], 'instruction': ['3원색이란 무엇인가요?', '어려운 결정을 내려야 했던 순간을 설명해 주세요.', '원자의 구조를 설명합니다.'], 'input': ['', '', ''], 'output': ['세 가지 기본 색은 빨강, 파랑, 노랑입니다. 이 색은 다른 색을 혼합하여 만들 수 없고 다른 모든 색은 다양한 비율로 조합하여 만들 수 있기 때문에 원색이라고 부릅니다. 빛에 사용되는 첨가제 색상 시스템에서 원색은 빨강, 녹색, 파랑(RGB)입니다.', 'AI 어시스턴트인 저는 저만의 개인적인 경험이나 물리적 존재를 가지고 있지 않습니다. 하지만 알고리즘, 논리, 데이터를 기반으로 의사 결정을 내리도록 프로그래밍되어 있습니다. 상충되는 정보나 여러 가지 가능한 결과가 있을 수 있기 때문에 의사결정이 어려운 상황에 직면할 때가 있습니다. 그런 경우 저는 각 옵션의 위험과 이점을 분석하고 비교하여 원하는 결과를 얻을 가능성이 가장 높은 결론에 도달할 수 있는 능력을 활용합니다.', '원자는 모든 물질의 기본 구성 요소이며 양성자, 중성자, 전자의 세 가지 유형의 입자로 이루어져 있습니다. 원자의 구조는 전자 구름으로 둘러싸인 중앙에 핵이 있는 것으로 설명할 수 있습니다.\n\n원자의 핵은 양성자와 중성자로 구성됩니다. 양성자는 양전하를 띠는 입자이고 중성자는 전하를 띠지 않는 중성 입자입니다. 이 두 입자는 원자의 중심에 있으며 원자 질량의 대부분을 차지하는 원자핵에 위치합니다.\n\n원자핵을 둘러싸고 있는 것은 전자 구름입니다. 전자는 음전하를 띠는 입자로, 원자핵 주변에서 끊임없이 움직입니다. 전자 구름은 껍질 또는 궤도로 나뉘며, 각 껍질은 특정 수의 전자를 보유할 수 있습니다. 원자가 껍질이라고 하는 가장 바깥쪽 껍질에 있는 전자의 수에 따라 원자의 화학적 특성이 결정됩니다.\n\n중성 원자에서 핵의 양성자 수는 전자 구름의 전자 수와 같으므로 양전

<br>

## 2. Tokenize
---
목적: 텍스트를 더 작은 부분, 즉 "토큰(token)"으로 분리하는 과정입니다.토큰화의 주요 목적은 텍스트 데이터를 단순화하는 것입니다. 긴 텍스트를 더 작은 단위로 나누어 알고리즘이 언어를 처리하고 이해하기 쉽게 만듭니다. 토큰은 언어를 이해하는 기반을 마련합니다. 이 토큰들을 분석함으로써, 알고리즘은 문장의 의미를 해석하고 단어가 사용된 맥락을 이해할 수 있습니다.
단위: 보통 단어, 문장, 문단 등을 토큰으로 분리합니다. 하지만 언어와 문맥에 따라 다를 수 있습니다.

### NLP에서 Tokenization

#### Tokenization (토큰화)

**정의**: 토큰화는 큰 문단을 문장, 단어 또는 다른 단위로 분해하는 과정입니다. 이 과정은 텍스트의 구조를 이해하는 데 필수적이며, 자연어 처리(NLP)에서 종종 첫 번째 단계 중 하나입니다.

**사용 사례**: 
- 텍스트 분류
- 감정 분석
- 기계 번역

**예시**: 
```text
입력: "안녕하세요, 세상! 어떻게 지내세요?"
토큰: ["안녕하세요", ",", "세상", "!", "어떻게", "지내세요", "?"]
```

**장점**:
- 단순하고 빠름
- 추가적인 텍스트 분석의 기초가 됨. 토큰화는 필터링, 정규화, 어간 추출(stemming) 및 표제어 추출(lemmatization) 같은 후속 데이터 전처리 단계의 기초를 제공합니다.
- 토큰화를 통해 언어의 구문론적(syntactic) 및 의미론적(semantic) 특성을 파악할 수 있습니다. 이는 품사 태깅, 개체명 인식 등의 고급 NLP 작업에 필수적입니다.
- 토큰화는 언어 모델이 텍스트를 효과적으로 이해하고 생성할 수 있게 도와줍니다. 이는 기계 번역, 감성 분석, 문서 요약 등의 작업에서 중요합니다.

**단점**:
- 단어 사이의 문맥과 의미 관계를 잃음
- 새롭게 생겨나는 신조어, 속어, 인터넷 용어 등은 기존의 토큰화 도구에서 제대로 인식되지 않을 수 있습니다.
- 단어들이 독립적으로 처리될 때 그 문맥상의 의미나 뉘앙스가 무시되기 쉽습니다.
- 관용 표현을 잘 처리하지 못할 수 있음

---

### tokenize 함수
- 주어진 텍스트(prompt)를 토큰화하는 역할을 합니다.
- 토큰화된 결과는 cutoff_len으로 정의된 최대 길이로 제한합니다.

### generate_and_tokenize_prompt 함수 
- 데이터를 바탕으로 프롬프트를 생성하고 토큰화하는 역할을 합니다.
- 학습 데이터를 모델에 적합한 형태로 변환하는 데 사용합니다.

In [6]:
def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point.get("input"),
        data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    if not train_on_inputs:
        user_prompt = prompter.generate_prompt(data_point["instruction"], data_point.get("input"))
        tokenized_user_prompt = tokenize(user_prompt, add_eos_token=add_eos_token)
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1

        tokenized_full_prompt["labels"] = [-100] * user_prompt_len + tokenized_full_prompt["labels"][
            user_prompt_len:
        ]  # could be sped up, probably
    return tokenized_full_prompt


In [7]:
dataset = data['train'].shuffle()#.select(range(100))
lm_dataset = dataset.map(generate_and_tokenize_prompt)

# Print total number of samples
print(f"Total number of samples: {len(lm_dataset)}")

Map:   0%|          | 0/152630 [00:00<?, ? examples/s]

Total number of samples: 152630


In [8]:
df_lm = lm_dataset.to_pandas()
df_lm

,id,instruction,input,output,input_ids,attention_mask,labels
0,vicuna_{idx},이 기본 예시를 더 정확한 모델로 바꾸려면 어떻게 해야 할까요?,,앞서 제공한 기본 몬테카를로 시뮬레이션 예제를 보다 정확한 모델로 전환하는 몇 가지...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ..."
1,dolly_{idx},지하철이란 무엇인가요?,지하 철도는 19세기 초에서 중반에 미국에 설립된 비밀 경로와 은신처 네트워크였습니...,지하 철도는 19세기 미국에서 노예들을 도와 자유를 찾게 해주는 비밀 경로와 은신처...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ..."
2,vicuna_{idx},eH는 갓 티어 광고의 창립자입니다.그의 유튜브 채널은 https://www.you...,,에드 리크에 대한 자세한 정보를 제공해 주셔서 감사합니다. 그의 유튜브 채널을 기반...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ..."
3,vicuna_{idx},마지막 장면을 작성합니다. 이 장면에서 마티 맥플라이와 닥 브라운은 집에 돌아와 매...,,INT. 맥플라이 하우스 - 밤마티와 닥은 소파에 앉아 맥주를 마시며 그들의 놀라운...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ..."
4,vicuna_{idx},"이 날 결혼식에 대한 긴 시를 쓰고 영원, 영원히, 폭포, 일출, 일몰, 은혜, 연...",,"칠칠절인 7일 아즈푸에서 결혼식이 열릴 예정이었어요,사랑과 은총으로 맺어진 두 사람...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ..."
...,...,...,...,...,...,...,...
152625,vicuna_{idx},설명과 함께 nodejs를 사용한 redisSearch,,edisSearch는 널리 사용되는 인메모리 키-값 저장소인 Redis를 기반으로 ...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ..."
152626,alpaca_{idx},이 텍스트를 두 개의 하위 섹션으로 나눕니다.,우리는 의료에서 금융에 이르기까지 의사 결정에 데이터가 점점 더 많이 활용되는 세상...,#### 하위 섹션 1: 의사 결정에 데이터 사용\n우리는 의료에서 금융에 이르기까...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ..."
152627,vicuna_{idx},캐나다의 기후 변화로 인한 식량 불안을 다루는 정책 개요를 작성하여 네 가지 정책 ...,,정책 요약: 캐나다의 기후 변화로 인한 식량 불안정 문제 해결소개:기후 변화는 캐나...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ..."
152628,vicuna_{idx},push\_update\_services를 여러 번 호출할 수 있습니다.,,f`push_update_services()`를 여러 번 호출해야 하고 각 호출이 ...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ..."


In [9]:
# 'input_ids' 칼럼의 각 벡터의 길이를 새로운 칼럼 'input_ids_length'에 저장
df_lm['input_ids_length'] = df_lm['input_ids'].apply(len)

# 새로운 칼럼을 확인
df_lm

,id,instruction,input,output,input_ids,attention_mask,labels,input_ids_length
0,vicuna_{idx},이 기본 예시를 더 정확한 모델로 바꾸려면 어떻게 해야 할까요?,,앞서 제공한 기본 몬테카를로 시뮬레이션 예제를 보다 정확한 모델로 전환하는 몇 가지...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...",567
1,dolly_{idx},지하철이란 무엇인가요?,지하 철도는 19세기 초에서 중반에 미국에 설립된 비밀 경로와 은신처 네트워크였습니...,지하 철도는 19세기 미국에서 노예들을 도와 자유를 찾게 해주는 비밀 경로와 은신처...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...",482
2,vicuna_{idx},eH는 갓 티어 광고의 창립자입니다.그의 유튜브 채널은 https://www.you...,,에드 리크에 대한 자세한 정보를 제공해 주셔서 감사합니다. 그의 유튜브 채널을 기반...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...",1011
3,vicuna_{idx},마지막 장면을 작성합니다. 이 장면에서 마티 맥플라이와 닥 브라운은 집에 돌아와 매...,,INT. 맥플라이 하우스 - 밤마티와 닥은 소파에 앉아 맥주를 마시며 그들의 놀라운...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...",580
4,vicuna_{idx},"이 날 결혼식에 대한 긴 시를 쓰고 영원, 영원히, 폭포, 일출, 일몰, 은혜, 연...",,"칠칠절인 7일 아즈푸에서 결혼식이 열릴 예정이었어요,사랑과 은총으로 맺어진 두 사람...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...",489
...,...,...,...,...,...,...,...,...
152625,vicuna_{idx},설명과 함께 nodejs를 사용한 redisSearch,,edisSearch는 널리 사용되는 인메모리 키-값 저장소인 Redis를 기반으로 ...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...",1399
152626,alpaca_{idx},이 텍스트를 두 개의 하위 섹션으로 나눕니다.,우리는 의료에서 금융에 이르기까지 의사 결정에 데이터가 점점 더 많이 활용되는 세상...,#### 하위 섹션 1: 의사 결정에 데이터 사용\n우리는 의료에서 금융에 이르기까...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...",242
152627,vicuna_{idx},캐나다의 기후 변화로 인한 식량 불안을 다루는 정책 개요를 작성하여 네 가지 정책 ...,,정책 요약: 캐나다의 기후 변화로 인한 식량 불안정 문제 해결소개:기후 변화는 캐나...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...",901
152628,vicuna_{idx},push\_update\_services를 여러 번 호출할 수 있습니다.,,f`push_update_services()`를 여러 번 호출해야 하고 각 호출이 ...,"[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[12657, 272, 2236, 276, 1251, 284, 272, 5026, ...",451


#### 토크나이징 벡터의 최대 길이는 설정한 cutoff_len로 고정됨

In [10]:
# 'input_ids_length' 칼럼의 최대값을 확인
max_value = df_lm['input_ids_length'].max()

print(f"The maximum length of 'input_ids' is {max_value}")

The maximum length of 'input_ids' is 2048


<br>

## 4. Save dataset to S3
---

In [11]:
import sagemaker
import boto3
sess = sagemaker.Session()
region = boto3.Session().region_name
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
bucket = None
if bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=bucket)

print(f"SageMaker role arn: {role}")
print(f"SageMaker bucket: {sess.default_bucket()}")
print(f"SageMaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/SageMaker/.xdg/config/sagemaker/config.yaml
SageMaker role arn: arn:aws:iam::057716757052:role/dt2gsmoon
SageMaker bucket: sagemaker-us-east-1-057716757052
SageMaker session region: us-east-1


In [12]:
import random
#  서로 다른 S3 위치에 저장하기 위해 S3 경로의 Prefix 생성
random_integer = random.randint(1, 100000)  # 1과 10 사이의 정수 난수 생성

bucket_prefix = f'ko-llms/peft/{random_integer}'
dataset_prefix = 'alpaca-train'
dataset_prefix_50_samples = 'alpaca-train-50-samples'
dataset_prefix_10000_samples = 'alpaca-train-1000-samples'
dataset_prefix_all = 'alpaca-train-all'

s3_data_path = f"s3://{bucket}/{bucket_prefix}/{model_name}/dataset/{dataset_prefix_all}"
s3_data_path_50_samples = f"s3://{bucket}/{bucket_prefix}/{model_name}/dataset/{dataset_prefix_50_samples}"
s3_data_path_10000_samples = f"s3://{bucket}/{bucket_prefix}/{model_name}/dataset/{dataset_prefix_10000_samples}"

s3_pretrained_model_path = f"s3://{bucket}/{bucket_prefix}/huggingface-models/{model_name}/"
print(f"S3 data path: \n {s3_data_path}")
print(f"S3 dataset_prefix_50_samples data path: \n {s3_data_path_50_samples}")
print(f"S3 dataset_prefix_10000_samples data path: \n {s3_data_path_10000_samples}")
print(f"S3 pretrained model path: \n {s3_pretrained_model_path}")

S3 data path: 
 s3://sagemaker-us-east-1-057716757052/ko-llms/peft/64903/kullm-polyglot-12-8b-v2/dataset/alpaca-train-all
S3 dataset_prefix_50_samples data path: 
 s3://sagemaker-us-east-1-057716757052/ko-llms/peft/64903/kullm-polyglot-12-8b-v2/dataset/alpaca-train-50-samples
S3 dataset_prefix_10000_samples data path: 
 s3://sagemaker-us-east-1-057716757052/ko-llms/peft/64903/kullm-polyglot-12-8b-v2/dataset/alpaca-train-1000-samples
S3 pretrained model path: 
 s3://sagemaker-us-east-1-057716757052/ko-llms/peft/64903/huggingface-models/kullm-polyglot-12-8b-v2/


### 로컬학습을 위한 데이터셋 저장
- dataset_prefix : 디버깅을 위한 처음 50개의 샘플(num_debug_samples)을 선택하여 dataset_prefix에 지정된 경로에 저장합니다. 
- dataset_prefix_all : 152630개를 모두 학습에 사용합니다. 

In [13]:
num_debug_samples = 50
num_short_train_samples = 10000

# save to local
lm_dataset.select(range(num_debug_samples)).save_to_disk(dataset_prefix)
# save to s3
lm_dataset.save_to_disk(s3_data_path)
lm_dataset.select(range(num_debug_samples)).save_to_disk(s3_data_path_50_samples)
lm_dataset.select(range(num_short_train_samples)).save_to_disk(s3_data_path_10000_samples)

lm_dataset.save_to_disk(dataset_prefix_all)
print(f"Number of samples for debugging: {num_debug_samples}")

Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


Saving the dataset (0/3 shards):   0%|          | 0/152630 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/152630 [00:00<?, ? examples/s]

Number of samples for debugging: 50


In [14]:
%store bucket_prefix dataset_prefix_50_samples s3_data_path s3_data_path_50_samples s3_data_path_10000_samples
%store dataset_prefix

Stored 'bucket_prefix' (str)
Stored 'dataset_prefix_50_samples' (str)
Stored 's3_data_path' (str)
Stored 's3_data_path_50_samples' (str)
Stored 's3_data_path_10000_samples' (str)
Stored 'dataset_prefix' (str)
